In [12]:
import pandas as pd
import numpy as np
import random
import os
from docx import Document
from docx.shared import Inches, Pt

# implementing random seed for control
sd = 42
np.random.seed(sd)

# school names for consistency 
schools = ['Oakland Middle School',
    'Siegel Middle School',
    'Whitworth-Buchanan Middle School',
    'Christiana Middle School',
    'Smyrna Middle School',
    'Stewarts Creek Middle School',
    'Rockvale Middle School',
    'Rocky Fork Middle School',
    'Blackman Middle School',
    'Thurman Francis Arts Academy',
    'Rock Springs Middle School',
    'LaVergne Middle School'
]

# preparing directories
for school in schools:
    path = f'../YouScienceData/Schedules/{school}'
    schedule_path = f'../YouScienceData/Schedules/{school}/Student_Schedules'
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(schedule_path):
        os.makedirs(schedule_path)


In [ ]:
""" 
Creates draft of per-student .docx schedule file, Round 1: Do for one school for proof of concept
FIXED in alpha1.ipynb:
    (1) Read in {school}_all_students.csv file
    (2) Read in Skyward_{school}.xlsx file
    (3) Create a 1st period/Homeroom feature
    (4) AZ by student name (should be default), then AZ by homeroom teacher (ease of school admin)
    (5) Begin writing draft schedule template
"""

In [16]:
# Date of YouScience Career Fair
date = '8 February 2023'

# Needed featuers: (1) student name, (2) Homeroom Teacher name, (3a-3d) Block Pathway title, 
# (4a-4d) Pathway Instructor, (5a-5d) Pathway room #, (6a-6d) Pathway zoom link

def build_student_schedules(school_df, planning_df):
    # iterate through students
    for i in range(len(school_df)):
        student_case = school_df.iloc[i]
        # get (1) student and (2) homeroom teacher names
        student_name, HR_teacher = student_case['name'], student_case['CY Teacher']

        # initialize schedule document & header format
        schedule_doc = Document()
        section = schedule_doc.sections[0]
        header = section.header
        head_para = header.paragraphs[0]
        head_para.text = f"{school}\tRutherford County School's Career Pathway Fair\t{date}"
        # text_run = head_para.add_run()
        # text_run.text = f'{school}\tYouScience Career Fair\t{date}'
        # text_run.style = "Heading 2 Char"

        # logo_run = head_para.add_run()
        # logo_run.add_picture("../../../../Desktop/QR_Career_Fair_2023.png", width=Inches(2))

        schedule_doc.add_heading(f'Name: {student_name}', level = 1)
        
        # schedule_doc.add_picture("../../../../Desktop/QR_Career_Fair_2023.png", width=Inches(1.5))
        schedule_doc.add_paragraph(f'Homeroom: {HR_teacher}')

        # build document body
        for block in ['B1','B2','B3','B4']: 
            # get pathway's info
            # (3a)
            pathway = student_case[block]
            POS_info = planning_df.loc[planning_df.POS == pathway]
            # (4a), (5a), (6a)
            POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
            POS_room_no = POS_info['MS Room #'].values[0]
            #POS_zoom_link = POS_info['Zoom Link'].values[0]

            # structure document
            schedule_doc.add_heading(f'Program of Study {block[1]}: {pathway}', level=3)
            schedule_doc.add_paragraph(
                f'Room #: {POS_room_no}, Sponsor/Instructors: {POS_instructors}'#\nZoom: {POS_zoom_link}'
            )
            # adding prompts per session
            prompts = ["What are 3 careers this program would help you explore?", 
                "What are some activities that you do in this Program of Study?",
                "Is this a program that you could see yourself enjoying?",
            ]
            # creating and populating table format for white space responses
            table = schedule_doc.add_table(rows=2, cols=3)
            row = table.rows[0].cells
            row[0].text = prompts[0]
            row[1].text = prompts[1]
            row[2].text = prompts[2]
            # for prompt in prompts:
            #     row = table.add_row().cells
            #     row[1].text = prompt
            #     row[1].width = Inches(2.5)
            #     row[2].width = Inches(2.0)

        name_for_path = '_'.join(student_name.split())
        if name_for_path == 'JORDIN/ALBUS_MCCLANAHAN':
            name_for_path = 'JORDIN_MCCLANAHAN'
        doc_path = f'../YouScienceData/Schedules/{school}/Student_Schedules/{name_for_path}.docx'
        schedule_doc.save(doc_path)

    return 

def Print_Schedules(school=None, date=date):

    if school == None:
        return 0

    # load in appropriate dfs
    school_path = f'../YouScienceData/Schedules/{school}/{school}_all_students.csv'
    school_df = pd.read_csv(school_path)
    school_df.drop('Unnamed: 0', axis=1, inplace=True)
    
    if school == 'Whitworth-Buchanan Middle School':
        planning_df = pd.read_excel('planning.xlsx', sheet_name='Whitworth-Buchanan Middle Schoo')
    else:
        planning_df = pd.read_excel('planning.xlsx', sheet_name=school)

    # execute schedule building
    build_student_schedules(school_df=school_df, planning_df=planning_df)
    print(f'{school} scheduls created: {len(school_df)}.')
    return 

for school in schools:
    Print_Schedules(school=school, date=date)
    break

Oakland Middle School scheduls created: 420.


## Troubleshooting

In [ ]:
p = f'../YouScienceData/Schedules/{school}/{school}_all_students.csv'
df = pd.read_csv(p)
df.drop('Unnamed: 0', axis=1, inplace=True)
planning_df = pd.read_excel('planning.xlsx', sheet_name=school)
df.head()

In [ ]:
case = df.iloc[4]
pathway = case.B2
POS_info = planning_df.loc[planning_df.POS == pathway]
POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])

In [ ]:
planning_df

In [ ]:
# troubleshooting... who put a '/' in a student's name?! 
school = 'Blackman Middle School'
p = f'../YouScienceData/Schedules/{school}/{school}_all_students.csv'
df = pd.read_csv(p)
df.drop('Unnamed: 0', axis=1, inplace=True)

df.loc[df.name == 'JORDIN/ALBUS MCCLANAHAN'] # Comes from Original YouScience doc... Not found anywhere else. wtf?! 

In [ ]:
# test case student
test_case = school_all_df.iloc[5]
# Needed featuers: (1) student name, (2) Homeroom Teacher name, (3a-3d) Block Pathway title, 
# (4a-4d) Pathway Instructor, (5a-5d) Pathway room #, (6a-6d) Pathway zoom link
# (1), (2)
student_name, HR_teacher = test_case['name'], test_case['CY Teacher']
# (3a) 
pathway = test_case.B1
POS_info = planning_df.loc[planning_df.POS == pathway]
# 4a, 5a, 6a
POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
POS_room_no = POS_info['MS Room #'].values[0]
POS_zoom_link = POS_info['Zoom Link'].values[0]

test_document = Document()
test_document.add_heading(f'Name: {student_name}', level = 1)
test_document.add_paragraph(f'Homeroom: {HR_teacher}')
test_document.add_heading(f'Pathway 1: {pathway}', level = 3)
test_document.add_paragraph(
    f'Sponsor/Instructors: {POS_instructors}\nRoom #: {POS_room_no}\nZoom: {POS_zoom_link}'
)

test_document.save('../test_schedule.docx')

In [ ]:
# Test version 2: adding for loop to get each pathway paragraph/heading
date = '8 February 2023'
test_case2 = school_all_df.iloc[10]
# (1), (2)
student_name, HR_teacher = test_case2['name'], test_case2['CY Teacher']
# Initializing document
test_document2 = Document()
section = test_document2.sections[0]
header = section.header
head_para = header.paragraphs[0]
head_para.text = f'{school}\tYouScience Career Fair\t{date}'
test_document2.add_heading(f'Name: {student_name}', level = 1)
test_document2.add_paragraph(f'Homeroom: {HR_teacher}')

for block in ['B1','B2','B3','B4']: 
    # get pathway's info
    # 3a
    pathway = test_case2[block]
    POS_info = planning_df.loc[planning_df.POS == pathway]
    # 4a, 5a, 6a
    POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
    POS_room_no = POS_info['MS Room #'].values[0]
    POS_zoom_link = POS_info['Zoom Link'].values[0]

    # structure document
    test_document2.add_heading(f'Pathway {block[1]}: {pathway}', level=3)
    test_document2.add_paragraph(
        f'Sponsor/Instructors: {POS_instructors}\nRoom #: {POS_room_no}\nZoom: {POS_zoom_link}'
    )

test_document2.save('../test_schedule3.docx')

In [ ]:
Print_Schedules(school=school, date=date)

### Editing, Checking POS/YS Clusters were translated correctly

In [ ]:
# Checking pathways are established correctly
planning_pos_list = list(planning_df.POS)
df_pos_list = list(school_all_df.B1.unique())
for pos in df_pos_list:
    if pos not in planning_pos_list:
        check = str(pos[:5])
        print(pos)
        for option in planning_pos_list:
            if check in option:
                if option in df_pos_list:
                    swap = 'Already in use.'
                else:
                    swap = 'Can switch!'
                print(school, option, swap)